# COMP9418 - Assignment 1 - Bayesian Networks as Classifiers

## UNSW Sydney, October 2020

- Student name 1 - zID
- Student name 2 - ZID

## Instructions

**Submission deadline:** Sunday, 18th October 2020, at 18:00:00.

**Late Submission Policy:** The penalty is set at 20% per late day. This is ceiling penalty, so if a group is marked 60/100 and they submitted two days late, they still get 60/100.

**Form of Submission:** This is a group assignment. Each group can have up to **two** students. **Only one member of the group should submit the assignment**.

You can reuse any piece of source code developed in the tutorials.

Submit your files using give. On a CSE Linux machine, type the following on the command-line:

``$ give cs9418 ass1 solution.zip``

Alternative, you can submit your solution via the [WebCMS](https://webcms3.cse.unsw.edu.au/COMP9418/20T3).

## Technical prerequisites

These are the libraries your are allowed to use. No other libraries will be accepted.

In [2]:
# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# Allowed libraries
import numpy as np
import pandas as pd
import scipy as sp
import heapq as pq
import matplotlib as mp
import math
from itertools import product, combinations
from collections import OrderedDict as odict
from graphviz import Digraph
from tabulate import tabulate

## Initial task - Initialise graph

Create a graph ``G`` that represents the following network by filling in the edge lists.
![Bayes Net](BayesNet.png)


In [11]:
G = {
    "BreastDensity" : ["Mass"],
    "Location" : ["BC"],
    "Age" : ["BC"],
    "BC" : ["Mass", "AD", "Metastasis", "MC", "SkinRetract","NippleDischarge"],
    "Mass" : ["Size",  "Shape", "Margin" ],
    "AD" : ["FibrTissueDev"],
    "Metastasis" : [ "LymphNodes"],
    "MC" : [],
    "Size" : [],
    "Shape" : [],
    "FibrTissueDev" : [ "SkinRetract" , "NippleDischarge","Spiculation" ],
    "LymphNodes" : [],
    "SkinRetract" : [],
    "NippleDischarge" : [],
    "Spiculation" : ["Margin" ],
    "Margin" : [],
}

## [20 Marks] Task 1 - Efficient d-separation test

Implement the efficient version of the d-separation algorithm in a function ``d_separation(G, X, Z, Y)`` that return a boolean: true if **X** is d-separated from **Y** given **Z** in the graph $G$ and false otherwise.

* **X**,**Y** and **Z** are python sets, each containing a set of variable names. 
* Variable names may be strings or integers, and can be assumed to be nodes of the graph $G$. 
* $G$ is a graph as defined in tutorial 1.

In [12]:
## Develop your code for d_separation(G, X, Z, Y) in one or more cells here
import copy 


def isleaf_node(G,node):
    return not G[node] 

def remove_leaf(G1,leaf_node):
    # delete the leaf node and its edges from the G, return a new Graph
    G_new = copy.deepcopy(G1)
    del G_new[leaf_node]
    for key, value in  G_new.items(): 
        if leaf_node in value:
            G_new[key].remove(leaf_node)
    return G_new
    
def repeat_del(G1,node_list): 
    count = 1 
    list_update = node_list.copy()
    while count > 0:
        count = 0
        for node in node_list: 
            if isleaf_node(G1,node):
                #print(node)
                #remove the nodes and update the graph 
                G1= copy.deepcopy(remove_leaf(G1,node))
                #print(G)
                list_update.remove(node)
                count = count + 1 
                #print(count)
        node_list = list_update.copy()
    return (G1)
    

    #if count == 0:
    #    return(G)
    #print(list_update)
    #new_list = list_update.copy()
    #G_new = repeat_del(G,new_list)
    
 
    
def dfs_r(G, v, colour):
    """
    argument 
    `G`, an adjacency list representation of a graph
    `v`, next vertex to be visited
    `colour`, dictionary with the colour of each node
    """
    #print('Visiting: ', v)
    # Visited vertices are coloured 'grey'
    colour[v] = 'grey'
    # Let's visit all outgoing edges from v
    for w in G[v]:
        # To avoid loops, we vist check if the next vertex hasn't been visited yet
        if colour[w] == 'white':
            dfs_r(G, w, colour)
    # When we finish the for loop, we know we have visited all nodes from v. It is time to turn it 'black'
    colour[v] = 'black' 
    return None
  


def d_separation(G1, X, Z, Y): 
    """ 
    Arguments: 
    `G`, an adjacency list representation of a graph 
    `X`, a set of variables name 
    `Y`, a set of variables name 
    `Z`, a set of a set of variables name 
    
    Returns 
    a boolean: true if X is d-separated from Y given Z in the graph  𝐺  and false otherwise.
    
    """
    if bool(X.intersection(Y).intersection(Z)):
        print("X, Y, Z are not disjoint")  #make it a warning/ error message? 
    
    combine_set = X.union(Y).union(Z)
    node_set = set(G1.keys())
    remain_nodes = set(node_set  - combine_set)
    
    G_final = copy.deepcopy(repeat_del(G1,remain_nodes)) #repeat del leaf nodes 
    
    for var in Z: 
        G_final[var] = [] #delete outgoing edges from Z set 
    
    for key,values in G_final.items():
        if bool(values):
            for n in values: 
                G_final[n].append(key) #make it undirect graph

    colour = {node: 'white' for node in G_final.keys()}
    #check connectivity 
    separate = True 
    for nodex in X:
        dfs_r(G_final,nodex,colour) 
        Y_color = [colour[node] for node in Y]
        #print(Y_color)
        if 'black' in Y_color:
            separate = False
            

    return(separate)
        

In [13]:
############
## TEST CODE

def test(statement):
    if statement:
        print("Passed test case")
    else:
        print("Failed test case")
        
test(d_separation(G, set(['Age']), set(['BC']), set(['AD'])))
test(not d_separation(G, set(['Spiculation','LymphNodes']), set(['MC', 'Size']), set(['Age'])))

Passed test case
Passed test case


## [10 Marks] Task 2 - Estimate Bayesian Network parameters from data

Implement a function ``learn_outcome_space(data)`` that learns the outcome space (the valid values for each variable) from the pandas dataframe ``data`` and returns a dictionary ``outcomeSpace`` with these values.

Implement a function ``learn_bayes_net(G, data, outcomeSpace)`` that learns the parameters of the Bayesian Network $G$. This function should return a dictionary ``prob_tables`` with the all conditional probability tables (one for each node).

- ``G`` is a directed acyclic graph. For this part of the assignment, $G$ should be declared according to the breast cancer Bayesian network presented in the diagram in the assignment specification.
- ``data`` is a dataframe created from a csv file containing the relevant data. 
- ``outcomeSpace`` is defined in tutorials.
- ``prob_tables`` is a dict from each variable name (node) to a "factor". Factors are defined in tutorial 2. 

In [25]:
## Develop your code for learn_outcome_space(data) in one or more cells here、
def learn_outcome_space(df):
    space_dict = {}
    var_list = list(df)
    for var in var_list:
        space_dict[var] = list(df[var].unique())
    return space_dict
    


In [26]:
############
## TEST CODE

with open('bc.csv') as file:
    data = pd.read_csv(file)

outcomeSpace = learn_outcome_space(data)

outcomes = outcomeSpace['BreastDensity']
answer = ('high', 'medium', 'low')
test(len(outcomes) == len(answer) and set(outcomes) == set(answer))

Passed test case


In [27]:
outcomeSpace

{'BreastDensity': ['high', 'medium', 'low'],
 'Location': ['LolwOutQuad', 'UpOutQuad', 'UpInQuad', 'LowInQuad'],
 'Age': ['35-49', '50-74', '>75', '<35'],
 'BC': ['No', 'Invasive', 'Insitu'],
 'Mass': ['No', 'Benign', 'Malign'],
 'AD': ['No', 'Yes'],
 'Metastasis': ['no', 'yes'],
 'MC': ['No', 'Yes'],
 'Size': ['<1cm', '1-3cm', '>3cm'],
 'Shape': ['Other', 'Oval', 'Round', 'Irregular'],
 'FibrTissueDev': ['No', 'Yes'],
 'LymphNodes': ['no', 'yes'],
 'SkinRetract': ['No', 'Yes'],
 'NippleDischarge': ['No', 'Yes'],
 'Spiculation': ['No', 'Yes'],
 'Margin': ['Well-defined', 'Ill-defined']}

In [39]:
## Develop your code for learn_bayes_net(G, data, outcomeSpace) in one or more cells here

def dict_copy(my_dict):
    new_dict={}
    for key,value in my_dict.items():
        new_dict[key] = value
    return new_dict

def graph_transpose(G):
    """
    argument 
    `G`, an adjacency list representation of a graph
    """      
    GT = dict((v, []) for v in G)
    for v in G:
        for w in G[v]:
            if w in GT:
                GT[w].append(v)
            else:
                GT[w] = [v]
    return GT

def prior_prob(var_list, df,outcome_space):
    """ return the probability table dictionary for nodes that is prior (no parents) , with category naming from 0 to n """
    new_oucome = dict_copy(outcome_space)
    prior_dict = {}
    for node in var_list:
        node_dict = {}
        node_dict['dom'] = tuple(new_oucome[node])
        
        prob = data[node].value_counts(normalize=True)
        prob_idx = list(prob.index)
        prob_value = list(prob)
        odict_list = [((i,), prob[i] )for i in prob_idx] 
        node_dict['table'] = odict(odict_list)
        prior_dict[node] = node_dict
    
    return prior_dict

In [40]:
junk = prior_prob(['Age'],data,outcomeSpace)

In [41]:
junk

{'Age': {'dom': ('35-49', '50-74', '>75', '<35'),
  'table': OrderedDict([(('50-74',), 0.50025),
               (('35-49',), 0.248),
               (('>75',), 0.1478),
               (('<35',), 0.10395)])}}

In [35]:
list(prob)

[50.025, 24.8, 14.78, 10.395]

In [36]:
prob['50-74']

50.025

In [ ]:
############
## TEST CODE

prob_tables = learn_bayes_net(G, data, outcomeSpace)
test(abs(prob_tables['Age']['table'][('35-49',)] - 0.2476) < 0.001)

## [20 Marks] Task 3 - Bayesian Network Classification

Design a new function ``assess_bayes_net(G, prob_tables, data, outcomeSpace, class_var)`` that uses the test cases in ``data`` to assess the performance of the Bayesian network defined by ``G`` and ``prob_tables``. Implement the efficient classification procedure discussed in the lectures. Such a function should return the classifier accuracy. 
 * ``class_var`` is the name of the variable you are predicting, using all other variables.
 * ``outcomeSpace`` was created in task 2
 
Remember to remove the variables ``metastasis`` and ``lymphnodes`` from the dataset before assessing the accuracy.

Return just the accuracy:

``acc = assess_bayes_net(G, prob_tables, data, outcomeSpace, class_var)``

In [2]:
## Develop your code for assess_bayes_net(G, prob_tables, data, outcomeSpace, class_var) in one or more cells here

In [ ]:
############
## TEST CODE

acc = assess_bayes_net(G, prob_tables, data, outcomeSpace, class_var)

Develop a function ``cv_bayes_net(G, data, class_var)`` that uses ``learn_outcome_space``, ``learn_bayes_net``and ``assess_bayes_net`` to learn and assess a Bayesian network in a dataset using 10-fold cross-validation. Compute and report the average accuracy over the ten cross-validation runs as well as the standard deviation, e.g.

``acc, stddev = cv_bayes_net(G, data, class_var)``

In [4]:
## Develop your code for cv_bayes_net(G, data, class_var) in one or more cells here

In [ ]:
############
## TEST CODE

acc, stddev = cv_bayes_net(G, data, 'BC')

## [10 Marks] Task 4 - Naïve Bayes Classification

Design a new function ``assess_naive_bayes(G, prob_tables, data, outcomeSpace, class_var)`` to classify and assess the test cases in a dataset ``data`` according to the Naïve Bayes classifier. To classify each example, use the log probability trick discussed in the lectures. This function should return the accuracy of the classifier in ``data``.

In [5]:
## Develop your code for assess_naive_bayes(G, prob_tables, data, outcomeSpace, class_var) in one or more cells here

In [ ]:
############
## TEST CODE

acc = assess_naive_bayes(G, prob_tables, data, outcomeSpace, 'BC')

Develop a new function ``cv_naive_bayes(data, class_var)`` that uses ``assess_naive_bayes`` to assess the performance of the Naïve Bayes classifier in a dataset ``data``. To develop this code, perform the following steps:

1. Use 10-fold cross-validation to split the data into training and test sets.

2. Implement a function ``learn_naive_bayes_structure(outcomeSpace, class_var)`` to create and return a Naïve Bayes graph structure from ``outcomeSpace`` and ``class_var``. 

3. Use ``learn_bayes_net(G, data, outcomeSpace)`` to learn the Naïve Bayes parameters from a training set ``data``. 

4. Use ``assess_naive_bayes(G, prob_tables, data, outcomeSpace, class_var)`` to compute the accuracy of the Naïve Bayes classifier in a test set ``data``. Remember to remove the variables ``metastasis`` and ``lymphnodes`` from the dataset before assessing the accuracy.

Do 10-fold cross-validation, same as above, and return ``acc`` and ``stddev``.

In [9]:
## Develop your code for learn_naive_bayes_structure(outcomeSpace, class_var) in one or more cells here

In [ ]:
############
## TEST CODE

naive_graph = learn_naive_bayes_structure(outcomeSpace, 'BC')

In [ ]:
## Develop your code for cv_naive_bayes(data, class_var) in one or more cells here

In [ ]:
############
## TEST CODE

acc, stddev = cv_naive_bayes(data, 'BC')

## [20 Marks] Task 5 - Tree-augmented Naïve Bayes Classification

Similarly to the previous task, implement a Tree-augmented Naïve Bayes (TAN) classifier and evaluate your implementation in the breast cancer dataset. Design a function ``learn_tan_structure(data, outcomeSpace, class_var)`` to learn the TAN structure (graph) from the ``data`` and returns such a structure.

In [7]:
## Develop your code for learn_tan_structure(data, outcomeSpace, class_var) in one or more cells here

In [ ]:
############
## TEST CODE

tan_graph = learn_tan_structure(data, outcomeSpace, class_var)
test(len(tan_graph['BC']) == len(tan_graph)-1)
test('FibrTissueDev' in tan_graph['Spiculation'] or 'Spiculation' in tan_graph['FibrTissueDev'])

Similarly to the other tasks, design a function ``cv_tan(data, class_var)`` that uses 10-fold cross-validation to assess the performance of the TAN classifier from ``data``. Remember to remove the variables ``metastasis`` and ``lymphnodes`` from the dataset before assessing the accuracy. This function should use the ``learn_tan_structure`` as well as other functions defined in this notebook.

In [8]:
## Develop your code for cv_tan(data, class_var) in one or more cells here

In [ ]:
############
## TEST CODE

acc, stddev = cv_tan(data, 'BC')

## [20 Marks] Task 6 - Report

Write a report (**with less than 500 words**) summarising your findings in this assignment. Your report should address the following:

a. Make a summary and discussion of the experimental results (accuracy). Use plots to illustrate your results.

b. Discuss the complexity of the implemented algorithms.

Use Markdown and Latex to write your report in the Jupyter notebook. Develop some plots using Matplotlib to illustrate your results. Be mindful of the maximum number of words. Please, be concise and objective.

In [ ]:
## Develop your report in one or more cells here